<a href="https://colab.research.google.com/github/bhanmrinal/NER-Chatbot/blob/main/Chatbot__LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Chatbot

This project aims to build a chatbot platform using Large Language Models. Tools used here include;

*   Hugging face model hub (to see models)
*   Python
*   Transformers (the brain box for our LLM)
*   Blenderbot by FacebookAI -LLM

LLM Use cases:
1.   Text generation
2.   Language translation
3.   Question & Answering
4.   Sentimental analysis
5.   Named entity recognition





In [ ]:
!pip install transformers

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
model_name = "facebook/blenderbot-400M-distill"

In [4]:
# Load model (download on first run and reference local installation for consequent runs)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

**Step 5a:** Keeping track of conversation history

The conversation history is important when interacting with a chatbot because the chatbot will also reference the previous conversations when generating output.

For our simple implementation in Python, we may simply use a **list**. Per the Hugging Face implementation, we will use this list to store the conversation history as follows:

`conversation_history`

`>> [input_1, output_1, input_2, output_2, ...]`

In [5]:
conversation_history = []

**Step 5b:** Encoding the conversation history

During each interaction, we **will pass our conversation history to the model** along **with our input so that it may also reference the previous conversation** when *generating the next answer*.

The transformers library function we are using expects to receive the conversation history as a string, with each element separated by the newline character '\n'. Thus, we create such a string.

We'll use the join() method in Python to do exactly that. (Initially, our history_string will be an empty string, which is okay, and will grow as the conversation goes on)

In [6]:
history_string = "\n".join(conversation_history)
history_string

''

In [47]:
input_text = "Hi I am Mrinal, a 19 year old student at DPS. My phone is 6006118448 and I live in Jammu. How are you?"

In [8]:
inputs = tokenizer.encode_plus(history_string, input_text, return_tensors="pt")
inputs

{'input_ids': tensor([[4424,  281,  632, 5501, 1692,   19,  265, 2202,  626,  983, 3841,  403,
         6986,   21,  863, 1782,  315, 1023,  619,   29, 3172,   31,   27,   27,
           31,  298,  281, 1248,  302,  587, 4332,   92,   21,  855,  366,  304,
           38]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [12]:
outputs = model.generate(**inputs)
outputs


tensor([[   1,  281,  476,  929,  731,   19, 2828,  304,  335, 2099,   21,  855,
          315,  587,  343,   92,   38,  281,  615,  716,  635,  505,   21,    2]])

In [48]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
response

'Sushi is my favorite as well. What is your favorite season? I love summer.'

In [14]:
conversation_history.append(input_text)
conversation_history.append(response)
conversation_history

['Hi I am Mrinal, a 19 year old student at DPS. My phone is 6006118448 and I live in Jammu. How are you?',
 "I'm doing well, thank you for asking. How is Jamu? I've never been there."]

In [ ]:
while True:
    # Create conversation history string
    history_string = "\n".join(conversation_history)

    # Get the input data from the user
    input_text = input("> ")

    # Tokenize the input text and history
    inputs = tokenizer.encode_plus(history_string, input_text, return_tensors="pt")

    # Generate the response from the model
    outputs = model.generate(**inputs)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Add interaction to conversation history
    conversation_history.append(input_text)
    conversation_history.append(response)

## **Deploying Chatbot to Backend Server**

In [ ]:
!pip install flask-ngrok
#when using flask on colab, not needed on local IDE

**Setup and Installation of Ngrok**

In [ ]:
# install ngrok linux version using the following command or you can get the
# latest version from its official website- https://dashboard.ngrok.com/get-started/setup

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

In [18]:
# extract the downloaded file using the following command

!tar -xvf /content/ngrok-stable-linux-amd64.tgz

ngrok


In [43]:
!./ngrok authtoken 2XW7ybcGXArXuCRqVuq1QoHhNHl_oLaMVUTYD9x9psYw1zGR

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


**Chatbot APP**

In [ ]:
pip install -U flask-cors

In [ ]:
from flask import Flask, request, render_template
from flask_cors import CORS
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from flask_ngrok import run_with_ngrok #for colab use only, remove in local IDE



app = Flask(__name__)
run_with_ngrok(app) #for colab use only, remove in local IDE
CORS(app)


model_name = "facebook/blenderbot-400M-distill"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
conversation_history = []

@app.route('/chatbot', methods=['POST'])
def handle_prompt():
    #read prompt from https request body
    data = request.get_data(as_text = True)
    data = json.loads(data)

    input_text = data['prompt'] # Get the input data from the user

    # Create conversation history string
    history_string = "\n".join(conversation_history)

    # Get the input data from the user #when testing prototype
    #input_text = input("> ")

    # Tokenize the input text and history
    inputs = tokenizer.encode_plus(history_string, input_text, return_tensors="pt")

    # Generate the response from the model
    outputs = model.generate(**inputs, max_length=60)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Add interaction to conversation history
    conversation_history.append(input_text)
    conversation_history.append(response)

    return response


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://772a-34-122-7-227.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 08:13:38] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 08:13:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 08:13:45] "GET /chatbot HTTP/1.1" 405 -
